# Compare commands from V2 versus V1

The V1 (legacy) commands archive is used as a reference standard to validate
the new V2 commands archive.

The primary difference is that V1 non-load commands and load stoppages are 
generated using the Chandra.cmd_states non-load commands table and timelines,
while V2 uses only the Flight Command Events from Google Sheets.

In [1]:
import sys
import os
from pathlib import Path
import difflib
import webbrowser

import numpy as np

# Using dev versions
sys.path.insert(0, '../')
sys.path.insert(0, str(Path.home() / 'git' / 'parse_cm'))
sys.path.insert(0, str(Path.home() / 'git' / 'testr'))

from kadi.commands import get_cmds, conf

In [2]:
from kadi import logger
# If needed for debugging...
# logger.setLevel(1)

### Check data prior to RLTT era (APR1420B)

In this era the commands are copied nearly verbatim, except:
- ``source`` column is added (using ``timeline_id``)
- ``timeline_id`` column is removed
- ``AOSTRCAT`` parameters are included

In [3]:
start = '2002-01-01'
stop = '2020-01-01'

with conf.set_temp('commands_version', '1'):
    cmds1 = get_cmds(start, stop)
with conf.set_temp('commands_version', '2'):
    cmds2 = get_cmds(start, stop)
    
assert len(cmds1) == len(cmds2)
assert np.all(cmds1['date'] == cmds2['date'])
assert np.all(cmds1['type'] == cmds2['type'])
assert np.all(cmds1['tlmsid'] == cmds2['tlmsid'])


### Generate diffs for commands from 2020-01-01 to 2022-01-01

In this era there are real diffs, but all are acceptable. The diffs have been
reviewed individually by aspect team members.

In [4]:
start = '2020-01-01'
stop = '2022-01-01'

In [5]:
with conf.set_temp('commands_version', '1'):
    cmds1 = get_cmds(start, stop)

In [6]:
with conf.set_temp('commands_version', '2'):
    cmds2 = get_cmds(start, stop)

In [7]:
# Commands V1 does not have starcat parameters or LOAD_EVENT commands
for cmds in cmds1, cmds2:
    ok = cmds['type'] == 'MP_STARCAT'
    cmds['params'][ok] = {}
    ok = cmds['type'] == 'LOAD_EVENT'
    cmds.remove_rows(np.where(ok)[0])
    cmds.sort(['date', 'step', 'scs'])

In [8]:
# There are differences in the number of commands
len(cmds1), len(cmds2)

(156600, 156617)

In [9]:
lines1 = cmds1.pformat_like_backstop(show_source=False, show_nonload_meta=False, max_params_width=40,
                                     sort_orbit_events=True)
lines2 = cmds2.pformat_like_backstop(show_source=False, show_nonload_meta=False, max_params_width=40, 
                                     sort_orbit_events=True)

In [10]:
htmldiffer = difflib.HtmlDiff()
html = htmldiffer.make_file(lines1, lines2, 'kadi commands v1', 'kadi commands v2', context=True)

out = Path('commands_v1_v2_diff.html')
out.write_text(html)

url = 'file://' + str(out.absolute())
webbrowser.open(url)

True